In [ ]:
import cv2
import json
import re
import subprocess

from datetime import datetime
from multiprocessing import Process, JoinableQueue as Queue
from os import listdir, path
from queue import Empty as QueueEmptyException

CAMERA_DB_PATH = "./metadata/cameras.json"

VIDEO_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/0801-500"
DB_PATH = "./metadata/videos.json"

In [ ]:
with open(DB_PATH, "r") as f:
  video_data = json.load(f)

with open(CAMERA_DB_PATH, "r") as f:
  camera_data = json.load(f)

In [ ]:
DATETIME_PATTERN = r'([0-9]{1,2})[-/]([0-9]{1,2})[-/]([0-9]{2,4}) ([0-9]{1,2}):([0-9]{1,2}):([0-9]{1,2})'
DATETIME_FORMAT = '%d%m%Y%H%M%S%z'

def string_to_epoch(datetime_string):
  matches = list(re.search(DATETIME_PATTERN, datetime_string).groups())
  matches = [('00'+m)[-2:] for m in matches]
  matches[2] = ('20'+matches[2])[-4:]
  with_utc_offset = "".join(matches) + "-0300"
  dt = datetime.strptime(with_utc_offset, DATETIME_FORMAT)
  return int(dt.timestamp())

In [ ]:
def get_times(vid, time_0, time_1):
  # if time_1 == time_0 or (time_1 - time_0) == (timestamp_1 - timestamp_0)
  
  return []

In [ ]:
to_process = Queue()

for io_dir in camera_data.keys():
  input_dir_path = path.join(VIDEO_PATH, io_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])

  for io_file in input_files:
    input_file_path = path.join(input_dir_path, io_file)

    if io_file not in video_data:
      video_data[io_file] = {
        "name": io_file,
        "camera": io_dir,
      }

    file_data = video_data[io_file]
    vid = None
    if not ("length_seconds" in file_data and "length_frames" in file_data):
      if vid is None:
        vid = cv2.VideoCapture(input_file_path)

      fps = vid.get(cv2.CAP_PROP_FPS)
      length_frames = vid.get(cv2.CAP_PROP_FRAME_COUNT)
      file_data["length_frames"] = length_frames

      vid.set(cv2.CAP_PROP_POS_AVI_RATIO, 1)
      length_seconds_seek = vid.get(cv2.CAP_PROP_POS_MSEC) // 1000
      length_seconds_fps = length_frames // fps

      if abs(length_seconds_fps - length_seconds_seek) > 2:
        print("ERROR: video length", io_file, length_seconds_fps, length_seconds_seek)
      else:
        file_data["length_seconds"] = length_seconds_fps

    if not ("time_start" in file_data and "time_end" in file_data):
      if vid is None:
        vid = cv2.VideoCapture(input_file_path)

      seek = []
      length_frames = vid.get(cv2.CAP_PROP_FRAME_COUNT)

      vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
      ret, frame_0 = vid.read()

      vid.set(cv2.CAP_PROP_POS_FRAMES, length_frames - 1)
      ret, frame_n = vid.read()

      # crop frames
      # process frames and get localtime strings
      # string_to_epoch(datetime_string)
      # set continuous

    if vid is not None:
      vid.release()

In [ ]:
with open(DB_PATH, "w") as f:
  json.dump(video_data, f, indent=2)